In [16]:
# Initialize PaddleOCR instance
from paddleocr import PaddleOCR
import cv2
import numpy as np

# paddleocr model for detection and extraction of text
ocr = PaddleOCR(
    use_angle_cls = False,
    lang='en',
    use_doc_orientation_classify=False,
    use_doc_unwarping=False,
    text_det_box_thresh=0,
)

C:\Users\hp333\AppData\Local\Temp\ipykernel_15484\3447655961.py:7: DeprecationWarning: The parameter `use_angle_cls` has been deprecated and will be removed in the future. Please use `use_textline_orientation` instead.
  ocr = PaddleOCR(
Creating model: ('PP-OCRv5_server_det', None)
Model files already exist. Using cached files. To redownload, please delete the directory manually: `C:\Users\hp333\.paddlex\official_models\PP-OCRv5_server_det`.
Creating model: ('en_PP-OCRv5_mobile_rec', None)
Model files already exist. Using cached files. To redownload, please delete the directory manually: `C:\Users\hp333\.paddlex\official_models\en_PP-OCRv5_mobile_rec`.


In [9]:
def needs_opening(img):
    if len(img.shape) == 3:
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    else:
        gray = img.copy()

    if len(np.unique(gray)) == 2:  # Already binary
        bw = gray
    else:
        _, bw = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

    num_labels, labels, stats, _ = cv2.connectedComponentsWithStats(bw, connectivity=8)
    if num_labels < 2:
        return False

    img_area = bw.shape[0] * bw.shape[1]

    small_objects = sum(
        1 for area in stats[1:, cv2.CC_STAT_AREA]  # Skip background (index 0)
        if area < (img_area * 0.0001)  # Less than 0.01% of image
    )
    noise_threshold = max(20, num_labels * 0.1)  # At least 20 or 10% of objects
    return small_objects > noise_threshold


def needs_closing(img):
    if len(img.shape) == 3:
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    else:
        gray = img.copy()
    edges = cv2.Canny(gray, 50, 150)

    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3, 3))
    grad = cv2.morphologyEx(gray, cv2.MORPH_GRADIENT, kernel)
    edge_density = np.count_nonzero(edges) / edges.size
    grad_std = np.std(grad)

    return edge_density > 0.15 or grad_std > 40


def needs_dilation(img):
    if len(img.shape) == 3:
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    else:
        gray = img.copy()

    if len(np.unique(gray)) == 2:
        bw = gray
    else:
        _, bw = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

    if np.mean(bw) > 127:
        bw = cv2.bitwise_not(bw)
    dist = cv2.distanceTransform(bw, cv2.DIST_L2, 5)
    text_pixels = dist[dist > 0]

    if len(text_pixels) == 0:  # No text found
        return False

    mean_stroke = np.mean(text_pixels)
    median_stroke = np.median(text_pixels)
    min_stroke = max(1.5, min(bw.shape) * 0.003)
    return mean_stroke < min_stroke or median_stroke < 1.0

In [10]:
def intelligent_dpi_adjustment(img):
    """
    Complete intelligent DPI adjustment for OCR.
    Combines multiple strategies for best results.
    """
    h, w = img.shape[:2]
    print(f"\n{'='*50}")
    print(f"Original dimensions: {w}x{h}")

    # Strategy 1: Check if image is reasonable size
    min_dimension = min(w, h)
    max_dimension = max(w, h)

    # Rule 1: Minimum size check (for very small images)
    if max_dimension < 800:
        scale = 800 / max_dimension
        print(f"Rule: Image too small, scaling up by {scale:.2f}x")

    # Rule 2: Maximum size check (for very large images)
    elif max_dimension > 4000:
        scale = 4000 / max_dimension
        print(f"Rule: Image too large, scaling down by {scale:.2f}x")

    # Rule 3: Optimal range (1200-2500px on longest edge)
    elif max_dimension < 1200:
        scale = 1500 / max_dimension
        print(f"Rule: Upscaling to optimal range ({scale:.2f}x)")
    elif max_dimension > 2500:
        scale = 2000 / max_dimension
        print(f"Rule: Downscaling to optimal range ({scale:.2f}x)")
    else:
        scale = 1.0
        print(f"Rule: Size already optimal, no scaling needed")

    # Apply scaling
    if scale != 1.0:
        new_w = int(w * scale)
        new_h = int(h * scale)

        # Choose best interpolation
        if scale > 1.0:
            interpolation = cv2.INTER_CUBIC  # Smooth upscaling
            print("Interpolation: INTER_CUBIC (upscaling)")
        else:
            interpolation = cv2.INTER_AREA  # Sharp downscaling
            print("Interpolation: INTER_AREA (downscaling)")

        img = cv2.resize(img, (new_w, new_h), interpolation=interpolation)
        print(f"Final dimensions: {new_w}x{new_h}")
        print(f"Equivalent DPI: ~{new_h / 11 * 300:.0f} DPI (assuming 11\" height)")

    print(f"{'='*50}\n")
    return img

In [14]:
#preprocessing pipeline
path = "C:/Users/hp333/Desktop/ocr/Data/Test Data/reverseWaybill-156387426414724544_1.jpg"
img = cv2.imread(path)

h, w = img.shape[:2]
print("height : ", h, " width : ", w)

gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
# img = cv2.adaptiveThreshold(gray_img, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C,cv2.THRESH_BINARY, 11, 2)
clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
img = clahe.apply(gray_img)

#smoothing
noise_level = np.std(img)
print("Noise level: ", noise_level)
if noise_level > 35:
    img = cv2.fastNlMeansDenoising(img, None, h=10, templateWindowSize=7, searchWindowSize=21)
    print("smoothing done")
elif noise_level > 25:
    img = cv2.GaussianBlur(img, (3, 3), 0)
    print("smoothing done")

# sharpening using kernels
laplacian_var = cv2.Laplacian(gray_img, cv2.CV_64F).var()
print("Laplacian variance:", laplacian_var)

# Adaptive threshold based on image size
img_size = w * h
adaptive_threshold = 100 * (img_size / 1000000)  # Scale with megapixels
adaptive_threshold = max(50, min(adaptive_threshold, 500))  # Clamp between 50-500

print(f"Adaptive blur threshold: {adaptive_threshold:.1f}")

if laplacian_var < adaptive_threshold:
    print("Sharpening applied")
    # Unsharp masking (gentler than direct kernel)
    gaussian = cv2.GaussianBlur(img, (0, 0), 2.0)
    img = cv2.addWeighted(img, 1.5, gaussian, -0.5, 0)

    # Or use your kernel with reduced intensity
    # kernel = np.array([[0, -1, 0], [-1, 5, -1], [0, -1, 0]]) / 1.5
    # img = cv2.filter2D(img, -1, kernel)

# laplacian_var = cv2.Laplacian(gray_img, cv2.CV_64F).var()
# print("laplacian var: ", laplacian_var)
# if laplacian_var < 100:
#     print("sharpening done")
#     kernal = np.array([[0, -1, 0], [-1, 5, -1], [0, -1, 0]])
#     img = cv2.filter2D(img, -1, kernal)

#morphological cleanup
kernel_size = max(3, min(w, h) // 200)
opening_kernel = np.ones((max(3, min(w,h)//300), max(3, min(w,h)//300)), np.uint8)
closing_kernel = np.ones((max(5, min(w,h)//150), max(5, min(w,h)//150)), np.uint8)
if needs_dilation(img):
    print("dilation done")
    img = cv2.dilate(img, kernel_size)

if needs_opening(img):
    print("opening done")
    img = cv2.morphologyEx(img, cv2.MORPH_OPEN, opening_kernel)

if needs_closing(img):
    print("closing done")
    img = cv2.morphologyEx(img, cv2.MORPH_CLOSE, closing_kernel)

#DPI adjustments
img = intelligent_dpi_adjustment(img)
img = cv2.cvtColor(img, cv2.COLOR_GRAY2RGB)

# Results
cv2.imshow("preprocessed image", img)
cv2.waitKey(0)
cv2.destroyAllWindows()

height :  2160  width :  3840
Noise level:  38.04420779780377
smoothing done
Laplacian variance: 106.84604515594019
Adaptive blur threshold: 500.0
Sharpening applied
opening done

Original dimensions: 3840x2160
Rule: Downscaling to optimal range (0.52x)
Interpolation: INTER_AREA (downscaling)
Final dimensions: 2000x1125
Equivalent DPI: ~30682 DPI (assuming 11" height)



In [17]:
# Run OCR inference on a sample image
result = ocr.predict(
    input = img)

In [18]:
# Visualize the results and save the JSON results
for res in result:
    res.save_to_img("output")
    res.save_to_json("output")


There is not input file name as reference for name of saved result file. So the saved result file would be named with timestamp and random number: `1764440911_7981`.
